This is a modified version of the following notebook: https://www.kaggle.com/hamzaghanmi/lgbm-hyperparameter-tuning-using-optuna

The main differences are the following:

Dummy encoding of the categorical variables
Modified ranges of the test options adn increased number of test rounds
Use of GPU

In [ ]:
import optuna
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
xgb.__version__

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test  = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')
test.head()

In [ ]:
columns = test.columns[1:]
categorical_cols=['cat'+str(i) for i in range(10)]
continous_cols=['cont'+str(i) for i in range(14)]
target = train['target']

In [ ]:
for e in categorical_cols:
    le = LabelEncoder()
    train[e]=le.fit_transform(train[e])
    test[e]=le.transform(test[e])

In [ ]:
def objective(trial,data=train,target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data[columns], target, test_size=0.2,random_state=42)
    
    param = {
        #'eval_metric': 'rmse', 
        "objective": "reg:squarederror",
        'random_state': 48,
        'n_estimators': 1000,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'gpu_id': 0,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.2, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
        'max_depth': trial.suggest_categorical('max_depth', [5,10,20,30]),

    }
    
    model = XGBRegressor(**param)  
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=10,verbose=False)

    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
'''%%time
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)'''

In [ ]:
params = {'reg_alpha': 0.48918981977195675,
 'reg_lambda': 0.23166250104970132,
 'colsample_bytree': 0.5257968778075015,
 'subsample': 0.8959010674808804,
 'learning_rate': 0.0278438302966696,
 'max_depth': 5,
 'random_state': 48,
 'n_estimators': 1000,
 'metric': 'rmse'}

In [ ]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5,random_state=48,shuffle=True)
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train,target):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=target[trn_idx],target[test_idx]
    model = XGBRegressor(**params)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=20,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1

In [ ]:
np.mean(rmse)

In [ ]:
sub['target']=preds
sub.to_csv('submission_xgb_1.csv', index=False)